Problem: Predicting Usage of a Video Game Research Server

Adiya Mittal

BACKGROUND INFORMATION

In recent years, the use of data science in gaming has become increasingly important, especially in large-scale multiplayer environments. Online games like Minecraft generate a vast amount of behavioral data as users interact with the environment, each other, and the game system itself. By analyzing this data, researchers and developers can optimize gameplay experiences, manage resources more effectively, and better understand user behavior patterns.

At the University of British Columbia (UBC), a research group led by Professor Frank Wood is conducting such a study using a dedicated Minecraft server. As players explore and interact in this shared world, their actions are continuously logged. However, managing the infrastructure to support this project poses significant logistical challenges. In particular, the research team must ensure that the server infrastructure (such as computing power and software licenses) can handle peak usage times, when many players may be active at once. Understanding when these high-demand periods occur is crucial for effective planning and resource allocation.

This brings us to the central aim of our project: predicting high-demand time windows, where the number of active players is likely to be large. By forecasting demand patterns, the team can make evidence-based decisions to scale their operations appropriately, target player recruitment, and prevent service disruptions.

The question I will address is this:

Can the hour of the day and the day of the week predict the number of active players in a given time window on the Minecraft server?

To answer this question I will be using the sessions.csv data that is provided to us. The sessions.csv data contains these following columns:
1. experience- The age of the player
2. subscribe- tells whether the person has subscribed to the game or not
3. hashedEmail- the email of the player
4. played_hours- how many hours has the players played the game
5. name- name of the player
6. gender- gender of the player
7. Age- age of the player

METHODS:
Prepocessing and exploratory data analysis
1. Import libraries and sessions.csv dataset from the internet.
2. Cleaned and tidied data to ake it usable, by converting the column start_time and end_time into date and time and I converted the time provied to us into Pacific time zone.
3. Added columns to the dataset like, start_hour, day_of_week, active_session_duration
4. Made various plots depending on the needs of questions, like finding the connection between day of the week and the duration of the active session.
5. Split the data into training and testing set and found the best K value.

FINDING THE BEST K VALUE:
1. Use recipe function to center and scale data
2. Perform the cross validation to train the classifier multiple times
3. Create the nearest neighbor model with tune()
4. Add recipe and model to workflow. Use the tune_grid() function to add the vfold_cv() and the range of K values.
5. Then testing out the best_k value on the validation set.

TESTING OUR CLASSIFIER

1. Made a new model specification for the best $k$ value chosen, combined with the recipe made earlier in a workflow, and fit the classifier to our training set.
2. Used predict on the testing set to evaluate the classifier’s predicition RMSE on data it hadn’t seen before.
3. Tested the RMSE of our classifier.

IMPORTING LIBRARIES AND PACKAGES

In [ ]:
#importing the libraries
#  Visualization
library(ggplot2)

# Data manipulation
library(tibble)
library(dplyr)
library(tidyr)
library(purrr)
library(readr)

# Modelling and Preprocessing
library(tidymodels)

# Data splitting and resampling
library(rsample) # train_test_split_equivalent
library(tune) # GridSearchCV equivalent
library(workflows) #make_pipeline equivalent
library(recipes) #make_column_transformer and StandardScaler equivalent

# Metrics
library(yardstick) # for mean squared error

#Time and Dates
library(lubridate)


#formatting graphs
options(repr.plot.width=12, repr.plot.height=6)

Step 2: Loading data in R

IMPORTING DATSET 

I used read_csv to import the players.csv and sessions.csv function.

In [ ]:
#loading data into R
players<-read_csv("players.csv")
players
sessions<- read_csv("sessions.csv")
sessions

CLEANING AND TIDYING DATA

First, I wanted t convert the start_time and end_time into a differnt forma, where the time is expressed in the 24 hour clock. This helps me to differentiate the time and to reduct the chances of R getting the confuse of whether I was talking about the morning time period or the evening time period.

Second, I wanted to convert the time into Pacific time, so that is it interpretable.

In [ ]:
# CONVERT START_TIME AND END_TIME TO DATETIME
sessions<-sessions|>
mutate( start_time= dmy_hm(start_time), end_time= dmy_hm(end_time))
sessions

In [ ]:
#CONVERT THE TIMEZONE FROM GMT TO PST
sessions <- sessions |>
  mutate(
    start_time = with_tz(force_tz(start_time, tzone = "GMT"), tzone = "America/Los_Angeles"),
    end_time = with_tz(force_tz(end_time, tzone = "GMT"), tzone = "America/Los_Angeles")
  )
sessions 


WRANGLING DATA

To understand what the data set sessions brought us in form information, I extracted multiple things from the data:
1. created an hourly time range
2. then counted how many active sessions were present in each hour of the day
3. extracting which day of the week were the session happening
4. and lastly seperately extracting the amonut of sessions that happened on each day of the weekday amd weekend.

In [ ]:
#Creating an hourly time range
hours<- seq(from= floor_date(min(sessions$start_time, na.rm = TRUE), unit= "hour"), 
            to = ceiling_date(max(sessions$end_time, na.rm = TRUE), unit= "hour"), 
            by="1 hour")
head(hours)

In [ ]:
#Count how many sessions are active during each hour
concurrent_sessions<-map_dfr(hours, function(h){
active_players <- sum(sessions$start_time <= h & sessions$end_time >= h)
tibble( time=h, active_sessions = active_players)})
head(concurrent_sessions)

In [ ]:
#Extracting day of week
concurrent_sessions <- concurrent_sessions|>
mutate(day_of_week =wday(time, label = TRUE, abbr = FALSE))
concurrent_sessions

In [ ]:
# Create weekday sessions
weekday_sessions <-concurrent_sessions|>
filter(day_of_week %in% c ("Monday", "Tuesday", "Wednesday", "Thursday", "Friday"))|>
mutate( day_of_end ="weekday")
head(weekday_sessions)

In [ ]:
# Create weekend sessions
weekend_sessions <- concurrent_sessions |>
  filter(day_of_week %in% c("Saturday", "Sunday")) |>
  mutate(day_or_end = "weekend")
head( weekend_sessions)

In [ ]:
# Calculate sessions duration 
sessions_durations <- sessions |>
  mutate(
    play_duration_minutes = as.numeric(difftime(end_time, start_time, units = "mins")),
    day_of_week = wday(start_time, label = TRUE, abbr = FALSE),
    start_hour = hour(start_time),
    start_time_of_day = hour(start_time) + minute(start_time) / 60
  ) |>
  filter(play_duration_minutes > 0)
head(sessions_durations)

LOOKING AT DIFFERENT VARIABLE USING DIFFERNT PLOTS:

1. Concurrent sessions by hour- This plot shows the relationship between the hour of the day (24 hour clock) and the number of active sessions during each hour. I presented ths in the form of a bar plot because I wanted to compare different number of sessions. We see that later hours of the day have more active sessions.

2. Day of the week by active sessions- This plot shows the relationship between the day of the week and the number of active sessions during each day. I presented ths in the form of a bar plot because I wanted to compare different number of sessions on each day. The conclusion can be made that Saturday and Sunday have most active sessions and are more busier that weekdays.

3. Concurrent sessions by day and hour- This plot shows the relationship between the day of the week,hour of the day and the number of active sessions during each day. I presented ths in the form of a bar plot because I wanted to compare different number of sessions on each day and each hour.

4. Start time vs duration- This plot shows the relationship between the start time of the active session and the duration of minutes it continues. I presented ths in the form of a scatter plot because of the comparison between two independent quantities.

5. Duration vs day of the week-  This plot shows the relationship between the day of the week and the duration of each session. I presented this in the form of a bar plot because I wanted to compare the durations on each day. The conclusion can be made that Saturday and Friday host longer duration session.

In [ ]:
#Show concurrent sessions by hour and the plot

concurrent_sessions<- concurrent_sessions|>
mutate(hour= hour(time))

total_sessions_plot<- ggplot(concurrent_sessions, aes(x= factor(hour), y= active_sessions))+
geom_bar(stat= "identity", fill="skyblue")+
labs(
title="Total Actve sessions by hour",
x="Hour of day",
y="Total Active sessions")+
theme_minimal()
total_sessions_plot

In [ ]:
# Day of the week exploratory plot

concurrent_sessions<- concurrent_sessions|>
mutate( day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))
                                                      
dayofweek_exploratory_plot<- ggplot(concurrent_sessions, aes(x= day_of_week, y= active_sessions))+
geom_bar(stat= "identity", fill="green")+
labs(
title="Total sessions by the day of week",
x="DAY OF WEEK",
y="Total Active sessions")+
theme_minimal()
dayofweek_exploratory_plot


In [ ]:
#Show concurrent sessions by hour and day of week

concurrent_sessions<- concurrent_sessions|>
mutate(
hour=hour(time),
day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))

Total_sessions_dayofweek <- ggplot(concurrent_sessions, aes(x=factor(hour), y= active_sessions))+
geom_bar(stat= "identity", fill="pink")+
facet_wrap(~ day_of_week, ncol=1)+
labs(
title="Total sessions by the by hour by day",
x="hour of day",
y="Total Active sessions")+
theme_minimal()
Total_sessions_dayofweek



In [ ]:
#Scatter plot of start_time_of_day vs. play_duration_minutes
sessions_durations <-sessions_durations |>
mutate(
 day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))
head(sessions_durations)

duration_scatter <- ggplot(sessions_durations , aes(x=start_time_of_day , y= play_duration_minutes ))+
geom_point( alpha= 0.4, color="red")+
facet_wrap(~ day_of_week, ncol=1)+
labs(
title="session duration in minutes by the time of day",
x="hour of day",
y="duration of session (minutes)")+
theme_minimal()
duration_scatter

In [ ]:
#Bar chart play duration minutes vs day of the week 
avg_duration <- sessions_durations|>
group_by(day_of_week)|>
summarize(avg_play_duration = mean ( play_duration_minutes, na.rm= TRUE))

duration_bar <- ggplot(avg_duration , aes(x= day_of_week  , y= avg_play_duration))+
geom_col( fill="purple")+
labs(
title="session duration in minutes by day of the week",
x="day of the week",
y="duration of session (minutes)")+
theme_minimal()
duration_bar

ANALYSIS OF HOURS REGARDLESS OF WEEK

SPLITTING OUR DATA INTO TRAINING AND TESTING SET

Before working on our model, I need to split our data into training and testing sets. 

Using initial_split to split our dataframe into 75% training and 25% testing, since it shuffles our data for us and ensures an constant proportion of each class is present in both. The 75-25 split allows us to train our model on as many data points as possible while also keeping enough data for effective testing later.

In [ ]:
#Training and testing split 
set.seed(2000)

split_obj<- initial_split( concurrent_sessions, prop= 0.75)

sessions_train <- training(split_obj)
sessions_test <- testing(split_obj)

x_train <- sessions_train|>
select(hour)
head(x_train)

y_train<- sessions_train|>
pull(active_sessions)
head(y_train)

x_test <- sessions_test|>
select(hour)
head(x_test)

y_test<- sessions_test|>
pull(active_sessions)
head(y_test)

In the above code, I split the data into a training set to build my model on, and a testing set to test it. Using initial_split allowed me to shuffle the data before splitting (removing bias and order) and stratify the data by diag so that an equal proportion of each is in each set.

FINDING THE BEST K VALUE

CREATING THE MODEL

To create our model, these steps will be followed:

Create a recipe that uses hour (predictor variable) to predict active sessions
   
Create a specification that uses k-nearest neighbors, with neighbors = tune() to perform cross-validation.

Split the data into 5 equally sized chunks, known as folds.

Create a workflow combining the first 2 steps, with tune_grid instead of fit.

Repeat for each validation set for values of 1 ≤ $k$ ≤ 49

In [ ]:
# KNN- regression
knn_recipe <- recipe(active_sessions ~ hour, data = sessions_train) |>
  step_scale(all_predictors())|>
  step_center(all_predictors())

# Model specification
knn_spec <- nearest_neighbor( mode = "regression", neighbors = tune() )|>
  set_engine("kknn")

# Workflow
knn_workflow <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_spec)

# Tuning grid: neighbors 1 to 49
knn_grid <- tibble(neighbors = 1:49)

# Resampling setup: 5-fold CV
set.seed(2000)
folds <- vfold_cv(sessions_train, v = 5)

# Tune the model
knn_tuned <- tune_grid( knn_workflow,
  resamples = folds,
  grid = knn_grid,
  metrics = metric_set(rmse) )

# Get best model
best_knn <- knn_tuned |>
 select_best("rmse")

# Finalize workflow with best number of neighbors
final_knn_workflow <- knn_workflow |>
  finalize_workflow(best_knn)

# Fit on full training data
final_knn_fit <- final_knn_workflow |>
  fit(data = sessions_train)

# Predict on test data
predictions <- predict(final_knn_fit, sessions_test) |>
  bind_cols(sessions_test)

# Calculate RMSE
rmse(predictions, truth = active_sessions, estimate = .pred)

In [ ]:
# best rmse
knn_tuned<-tune_grid(knn_workflow, resamples=folds, grid=knn_grid, metrics=metric_set(rmse))
y_train<-pull(sessions_train, active_sessions)
y_train

In [ ]:
sessions_test_preds<- predict(final_knn_fit, new_data=sessions_test)|>
bind_cols(sessions_test)

#view and predict RMSPE results
rmse_result<-rmse(data=sessions_test_preds, truth=active_sessions, estimate=.pred)
rmse_result$.estimate

In [ ]:
# predict on training data
sessions_train_preds<- predict(final_knn_fit, new_data=sessions_train)|>
bind_cols(sessions_train)

ANALYSIS SEPERATING WEEKENDS AND WEEKDAYS

This code performs K-Nearest Neighbors (KNN) regression to model and predict the number of active sessions (active_sessions) based on the hour of the day (hour) from the weekday_sessions dataset. First, a random seed is set for reproducibility, and the hour is extracted from a time column using the lubridate::hour() function. The dataset is then split into a training set (75%) and a testing set (25%) using initial_split(). A grid search is conducted over values of k from 1 to 49 using 5-fold cross-validation to find the value that minimizes RMSE (Root Mean Square Error). The best value of k is selected, and the final model is retrained on the full training set.

In [ ]:
# split weekday sessions 
set.seed(2000)  # Reproducibility

# 75/25 split

weekday_sessions <- weekday_sessions |>
  mutate(hour = lubridate::hour(time))

weekday_split <- initial_split(weekday_sessions, prop = 0.75)
weekday_train <- training(weekday_split)
weekday_test <- testing(weekday_split)
weekday_train
weekday_test

weekday_recipe <- recipe(active_sessions ~ hour, data = weekday_train) |>
step_scale(all_predictors())|>
step_center(all_predictors())

weekday_knn_spec <- nearest_neighbor( mode = "regression", neighbors = tune()) |>
set_engine("kknn")

weekday_workflow <- workflow() |>
  add_recipe(weekday_recipe) |>
  add_model(knn_spec)

knn_grid <- tibble(neighbors = 1:49)

weekday_resamples <- vfold_cv(weekday_train, v = 5)

weekday_tuned <- tune_grid(
  weekday_workflow,
  resamples = weekday_resamples,
  grid = knn_grid,
  metrics = metric_set(rmse))

best_k <- select_best(weekday_tuned, "rmse")

weekday_final_wf <- finalize_workflow(weekday_workflow, best_k)

weekday_fit <- fit(weekday_final_wf, data = weekday_train)

weekday_test_preds <- predict(weekday_fit, new_data = weekday_test) |>
  bind_cols(weekday_test)

# Calculate RMSE
rmse(weekday_test_preds, truth = active_sessions, estimate = .pred)

weekday_train_preds <- predict(weekday_fit, new_data = weekday_train) |>
  bind_cols(weekday_train)

This code performs K-Nearest Neighbors (KNN) regression to model and predict the number of active sessions (active_sessions) based on the hour of the day (hour) from the weekend_sessions dataset. First, a random seed is set for reproducibility, and the hour is extracted from a time column using the lubridate::hour() function. The dataset is then split into a training set (75%) and a testing set (25%) using initial_split(). A grid search is conducted over values of k from 1 to 49 using 5-fold cross-validation to find the value that minimizes RMSE (Root Mean Square Error). The best value of k is selected, and the final model is retrained on the full training set.

In [ ]:
# for the weekends

weekend_sessions <- weekend_sessions |>
  mutate(hour = lubridate::hour(time))

weekend_split <- initial_split(weekend_sessions, prop = 0.75)
weekend_train <- training(weekend_split)
weekend_test <- testing(weekend_split)
weekend_train
weekend_test

weekend_recipe <- recipe(active_sessions ~ hour, data = weekend_train) |>
step_scale(all_predictors())|>
step_center(all_predictors())

weekend_knn_spec <- nearest_neighbor( mode = "regression", neighbors = tune()) |>
set_engine("kknn")

weekend_workflow <- workflow() |>
  add_recipe(weekend_recipe) |>
  add_model(knn_spec)

knn_grid <- tibble(neighbors = 1:49)

weekend_resamples <- vfold_cv(weekend_train, v = 5)

weekend_tuned <- tune_grid(
  weekend_workflow,
  resamples = weekend_resamples,
  grid = knn_grid,
  metrics = metric_set(rmse))

best_k <- select_best(weekend_tuned, "rmse")

weekend_final_wf <- finalize_workflow(weekend_workflow, best_k)

weekend_fit <- fit(weekend_final_wf, data = weekend_train)

weekend_test_preds <- predict(weekend_fit, new_data = weekend_test) |>
  bind_cols(weekend_test)

# Calculate RMSE
rmse(weekend_test_preds, truth = active_sessions, estimate = .pred)

weekend_train_preds <- predict(weekend_fit, new_data = weekend_train) |>
  bind_cols(weekend_train)

In [ ]:
# plot : maximumu concurrent seesions by the day of week

# Compute max active sessions per day_of_week
max_sessions <- concurrent_sessions %>%
  group_by(day_of_week) %>%
  summarise(max_active_sessions = max(active_sessions, na.rm = TRUE)) %>%
  ungroup() %>%
  mutate(day_of_week = factor(day_of_week,
                             levels = c('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')))

# Plot
maxconcurrent_weekday<-ggplot(max_sessions, aes(x = day_of_week, y = max_active_sessions)) +
  geom_col(fill = "steelblue") +
  labs(title = "Maximum Concurrent Sessions by Day of Week",
       x = "Day of Week",
       y = "Maximum Active Sessions Recorded") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))
maxconcurrent_weekday


The code below visualizes the relationship between the hour of the day and the number of active sessions using KNN regression predictions. First, it uses the trained final KNN model (final_knn_fit) to predict the number of active sessions for each row in the training data (sessions_train). These predictions are combined with the original data using bind_cols(). Then, it counts how many observations occur at each hour using count(hour), storing the result in count_data, and merges this count back into the predictions dataframe to use as a visual cue in the plot.

In [ ]:
# active sessions per hour
# calculate sessions preds
sessions_preds<-predict(final_knn_fit, new_data=sessions_train)|>
bind_cols(sessions_train)

# Calculate counts for each hour to size points
count_data <- sessions_preds|>
  count(hour)

# Merge counts back to sessions_preds
sessions_preds_counts <- sessions_preds|>
  left_join(count_data, by = "hour")

# Plot
sessions_data_plot<-ggplot(sessions_preds_counts, aes(x = hour, y = active_sessions)) +
  geom_point(aes(size = n), alpha = 0.4, shape = 16, fill = "black") +
  geom_line(aes(y = .pred), color = "blue") +
  scale_size_continuous(name = "Count") +
  labs(title = "Active Sessions Predictions by Time of Day",
       x = "Hour of Day",
       y = "Active Sessions") +
  theme_minimal() +
  theme(legend.position = "right")
sessions_data_plot

DISCUSSION:

WHAT WE WERE SUPPOSED TO FIND:
Our project set out to investigate when peak usage occurs on a gaming platform by analyzing session data, with the goal of forecasting demand—specifically, identifying time windows with high numbers of concurrent users. This helps determine how many game licenses need to be available at different times to meet demand without over-provisioning.

SUMMARY OF FINDINGS

DAY OF THE WEEK TRANDS:
The bar plot above shows the maximum number of concurrent sessions recorded for each day of the week. Sunday and Thursday stand out with the highest peak concurrent sessions, reaching 6. Saturday also shows elevated usage. Wednesday has the lowest peak, indicating lighter midweek traffic. This strongly suggests a weekend effect, with heavier usage on days when users are more likely to be off school or work. The high Thursday value may reflect users starting their weekend gaming earlier.

HOURLY SESSION TRENDS:
The dot plot above illustrates the distribution of active sessions by hour of day, with dot size indicating how frequently certain session levels occur. Very low activity is observed during early morning hours (12 AM–8 AM). Activity begins climbing in the late morning, rises steadily into the afternoon, and peaks between 6 PM and 10 PM. The single highest activity is centered around 8 PM, aligning with common evening leisure hours. The blue line overlay helps highlight the average trend, reinforcing that evenings are consistently the busiest period regardless of day.

SESSION DURATION TRENDS:
We also examined how long sessions last during different parts of the day: Longer gameplay sessions tend to occur in the afternoon and evening, when users have more uninterrupted time. Weekends (especially Saturday and Sunday) feature not only more sessions, but longer ones, suggesting immersive play. This finding helps further justify targeting infrastructure scaling around these time windows.

AVERAGE DURATION BY THE DAY:
Although not shown in the images above, we also analyzed average session duration per day: Friday and Saturday had the longest average play times.
Sunday had many sessions, but they were slightly shorter—possibly due to users preparing for the week ahead.



WAS THIS EXPECTED?

Given that people usually go to sleep between midnight and the early hours of the morning, it makes natural that user activity would be quite low during the night. Usage naturally increases as more people log in during the day. The late afternoon and evening peak, particularly between 6 and 10 p.m., is in line with typical patterns of digital usage, when people use entertainment platforms after work or school. Additionally, we anticipated that the time of day by itself would be a powerful indicator of user activity, and it was. However, we discovered that the hour of the day alone was insufficient to adequately account for all usage patterns. For instance, on some days, particularly, the KNN model understated some of the hours of high demand. This points to some underfitting, as the model was unable to adequately represent the intricacy of user behavior based solely on time. It supported our hypothesis that other factors, such as the day of the week, are also quite important. We therefore expected that treating weekdays and weekends differently would be crucial, and this proved to be the case. We saw glaring disparities as soon as we began breaking down the data by day. Weekday usage patterns were more moderate and dispersed, but weekend usage peaks were more intensive and concentrated. We were better able to comprehend how user involvement changes based on when people are most likely to have spare time thanks to this distinction.


IMPACT OF FINDINGS:

These findings can have direct operational impact:

License Planning: Knowing peak usage hours allows organizations to provision just enough licenses to meet demand without unnecessary over-purchasing.

Scheduling: Staff or system resources (e.g., load balancing) can be optimized around high-traffic time windows.

Monitoring & Scaling: Predictive models could be used in real time to trigger alerts or auto-scaling behaviors in digital environments.

REFERENCES:

1. Get date-time in a different time zone — with_tz. (2025). Tidyverse.org. https://lubridate.tidyverse.org/reference/with_tz.html

2. Guide to Logical Operators in R (OR, NOT and AND in R) | Built In. (n.d.). Builtin.com. https://builtin.com/data-science/and-in-r

‌

‌
